In [86]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [87]:
f = r'/home/william/Dropbox/Kaggle/Soccer/database.sqlite'
conn = sqlite3.connect(f)

In [88]:
matches = pd.read_sql('SELECT * from Match',conn)
leagues = pd.read_sql('SELECT * from League',conn)
teams = pd.read_sql('SELECT * from Team',conn)

In [89]:
teams.loc[teams['team_api_id']==8455,'team_long_name']

43    Chelsea
Name: team_long_name, dtype: object

In [90]:
pl = matches.iloc[1729:4269,[3, 4, 5, 7, 8, 9, 10, 85, 86, 87]]

In [91]:
season09 = pl[pl['season']=='2009/2010']

In [92]:
season09.head()

,season,stage,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A
2108,2009/2010,1,2009-08-16 00:00:00,10260,8658,1,0,1.20,6.00,17.0
2109,2009/2010,1,2009-08-16 00:00:00,8586,8650,2,1,3.20,3.25,2.3
2110,2009/2010,1,2009-08-15 00:00:00,10252,8528,0,2,1.67,3.60,5.5
2111,2009/2010,1,2009-08-15 00:00:00,8668,9825,1,6,3.20,3.25,2.3
2112,2009/2010,1,2009-08-15 00:00:00,8602,8654,0,2,2.63,3.30,2.7


In [93]:
def rinse_df(season09):
    season09 = season09.sort_values('date')

    home_win = np.where(season09['home_team_goal'] > season09['away_team_goal'], 1, 0)
    away_win = np.where(season09['home_team_goal'] < season09['away_team_goal'], 1, 0)
    season09['Home/Away win'] = home_win-away_win

    season09['points_home'] = 0
    season09.loc[season09['Home/Away win']==1, 'points_home'] = 3
    season09.loc[season09['Home/Away win']==0, 'points_home'] = 1

    season09['points_away'] = 0
    season09.loc[season09['Home/Away win']==-1, 'points_away'] = 3
    season09.loc[season09['Home/Away win']==0, 'points_away'] = 1

    season09['goal_difference'] = season09['home_team_goal'] - season09['away_team_goal']

    season09 = season09.reset_index(drop=True)

    team_nos = np.unique(season09['home_team_api_id'])
    no_matches = len(season09)

    scores = np.zeros((2*no_matches//len(team_nos),team_nos.size))

    for n in range(len(team_nos)):
        no = team_nos[n]
        for i in range(no_matches):
            row = season09.iloc[i]
            stage = row['stage']-1
            if row['home_team_api_id'] == no:
                scores[stage,n] = row['points_home']

            if row['away_team_api_id'] == no:
                scores[stage,n] = row['points_away']
    scores = np.cumsum(scores,axis=0)
    scores = np.vstack((np.zeros(len(team_nos)),scores))

    scores = np.copy(scores[:-1,:])

    season09['current_home'] = 0
    season09['current_away'] = 0
    for i in range(no_matches):
        row = season09.iloc[i]
        home_team = row['home_team_api_id']
        away_team = row['away_team_api_id']

        home_ind = np.where(team_nos==home_team)[0][0]
        away_ind = np.where(team_nos==away_team)[0][0]

        stage = row['stage']-1


        season09.loc[i,'current_home'] = scores[int(stage),int(home_ind)]
        season09.loc[i,'current_away'] = scores[int(stage),int(away_ind)]


    #A score for each time indicating relative competence
    #Rethink what the formula should be, given pseudorandom match result with bias
    season09['competence_difference'] = (season09['current_home']-season09['current_away']) / season09['stage']
    season09_relevant = season09.iloc[:,[1,7,8,9,10,-1]]
    return season09_relevant

In [192]:
#Creating season09-like dataframes from http://www.football-data.co.uk/
def extract_df(f):
    df = pd.read_csv(f)
    new_df = df.iloc[:,[1,2,3,4,5,23,24,25]]

    new_df.columns = ["date", "home_team_api_id", "away_team_api_id", 
                      "home_team_goal", "away_team_goal", "B365H", "B365D",
                      "B365A"]
    new_df.insert(0,"stage",0)
    new_df.insert(0,"season",0)

    new_df['date'] = pd.to_datetime(new_df['date'])
    new_df = new_df.sort_values("date")

    new_df.reset_index(inplace=True,drop=True)
    new_df['stage'] = new_df.index//10+1
    
    factorized = pd.factorize(new_df['home_team_api_id'])
    new_df['away_team_api_id'] = new_df['away_team_api_id'].replace(factorized[1],np.arange(20))
    new_df['home_team_api_id'] = factorized[0]
    
    return new_df

In [204]:
season09 = pl[pl['season']=='2009/2010']
season10 = pl[pl['season']=='2010/2011']
season11 = pl[pl['season']=='2011/2012']
season12 = pl[pl['season']=='2012/2013']
season13 = pl[pl['season']=='2013/2014']

season14 = extract_df('/home/william/Dropbox/Kaggle/Soccer/data14.csv')
season15 = extract_df('/home/william/Dropbox/Kaggle/Soccer/data15.csv')
season16 = extract_df('/home/william/Dropbox/Kaggle/Soccer/data16.csv')
season17 = extract_df('/home/william/Dropbox/Kaggle/Soccer/data17.csv')


season09 = rinse_df(season09)
season10 = rinse_df(season10)
season11 = rinse_df(season11)
season12 = rinse_df(season12)
season13 = rinse_df(season13)
season14 = rinse_df(season14)
season15 = rinse_df(season15)
season16 = rinse_df(season16)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ValueError: Replacement lists must match in length. Expecting 10 got 20 

In [202]:
season16.head()

,stage,B365H,B365D,B365A,Home/Away win,competence_difference
0,1,2.50,3.25,3.20,0,0.0
1,1,8.00,4.75,1.45,-1,0.0
2,1,2.38,3.30,3.30,0,0.0
3,1,2.25,3.40,3.50,0,0.0
4,1,7.50,4.50,1.50,-1,0.0


In [203]:
season09.to_csv('season9.csv',index=False)
season10.to_csv('season10.csv',index=False)
season11.to_csv('season11.csv',index=False)
season12.to_csv('season12.csv',index=False)
season13.to_csv('season13.csv',index=False)
season14.to_csv('season14.csv',index=False)
season15.to_csv('season15.csv',index=False)
season16.to_csv('season16.csv',index=False)


In [ ]:
#14: 100% profits at 82% of the data
#15: 57% profits at 78% of the data
#16: 150% profits at 81% of the data